In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Blank Correction & Lipid Nomalization

In [ ]:
tagToId = pd.read_excel('muslce,fin clips dioxcin,furans.xlsx')
finclip = pd.read_excel('muslce,fin clips dioxcin,furans.xlsx', sheet_name= 'Fin clips')
rl_nd = finclip.loc[finclip.iloc[:, -6] == "ND", finclip.columns[-4]].dropna()
finclip.loc[rl_nd.index, finclip.columns[-5]] = rl_nd.to_numpy('float32')/np.sqrt(2)

c = finclip.columns
c = c[~c.str.contains('Unnamed')]
finclip = finclip.iloc[5:, [0]].join(finclip.iloc[5:, 2::3])
finclip.columns = c.astype('str')
id_to_name_dict = dict(zip(tagToId['Tag name'], tagToId['Fish ID']))
id_to_name_dict.update({"13-Trembleur":"13"})
finclip.rename(columns=id_to_name_dict, inplace=True)
finclip.insert(1, 'tag', 'finclip')
finclip.columns = finclip.columns.map(lambda x: str(x))
lipid_id = finclip.iloc[:, 0].str.contains('lipid', case=False)
id = finclip.loc[~lipid_id, finclip.columns[-2]].dropna().index
finclip.loc[id, finclip.columns[2:-2]] -= finclip.loc[id, finclip.columns[-2]].to_numpy('float')[:, np.newaxis]
finclip = finclip.iloc[:, :-2]
print(f'count of samples corrected to negative: {np.sum((~(finclip.iloc[:, 2:].isna() | (finclip.iloc[:, 2:] > 0)).to_numpy()))}')
print(f'position of corrected samples: {np.where((~(finclip.iloc[:, 2:].isna() | (finclip.iloc[:, 2:] > 0))).to_numpy())}')
finclip.where((finclip.iloc[:, 2:]>0) | (finclip.iloc[:, 2:].isna()), 0, inplace = True)
finclip

finclip_l = finclip.copy()

lipid_id = np.arange(len(finclip_l.CLIENT_ID))[finclip_l.CLIENT_ID.str.contains('Lipid', case = False)][0]
finclip_l.iloc[lipid_id, 2:] /= 100
id = list(range(finclip_l.shape[0]))
id.pop(lipid_id)
finclip_l.iloc[id, 2:] /= finclip_l.iloc[lipid_id, 2:]
finclip_l

In [ ]:
muscle = pd.read_excel('muslce,fin clips dioxcin,furans.xlsx', sheet_name= 'Muslce')
rl_nd = muscle.loc[muscle.iloc[:, -6] == "ND", muscle.columns[-4]].dropna()
muscle.loc[rl_nd.index, muscle.columns[-5]] = rl_nd.to_numpy('float32')/np.sqrt(2)

c = muscle.columns
c = c[~c.str.contains('Unnamed')]
muscle = muscle.iloc[5:, [0]].join(muscle.iloc[5:, 2::3])
c = list(c.str.removeprefix("Nechako mort #"))
muscle.columns = c
muscle.insert(1, 'tag', 'muscle')

lipid_id = muscle.iloc[:, 0].str.contains('lipid', case=False)
id = muscle.loc[~lipid_id, muscle.columns[-2]].dropna().index
muscle.loc[id, muscle.columns[2:-2]] -= muscle.loc[id, muscle.columns[-2]].to_numpy('float')[:, np.newaxis]
muscle = muscle.iloc[:, :-2]
print(f'count of samples corrected to negative: {np.sum((~(muscle.iloc[:, 2:].isna() | (muscle.iloc[:, 2:] > 0)).to_numpy()))}')
print(f'position of corrected samples: {np.where((~(muscle.iloc[:, 2:].isna() | (muscle.iloc[:, 2:] > 0))).to_numpy())}')
muscle.where((muscle.iloc[:, 2:]>0) | (muscle.iloc[:, 2:].isna()), 0, inplace = True)


muscle_l = muscle.copy()

lipid_id = np.arange(len(muscle_l.CLIENT_ID))[muscle_l.CLIENT_ID.str.contains('Lipid', case = False)][0]
muscle_l.iloc[lipid_id, 2:] /= 100
id = list(range(muscle_l.shape[0]))
id.pop(lipid_id)
muscle_l.iloc[id, 2:] /= muscle_l.iloc[lipid_id, 2:]
muscle_l

blankcorrect_table = pd.concat([finclip, muscle], axis = 0)

blankcorrect_table['CLIENT_ID'] = blankcorrect_table["CLIENT_ID"].astype("category")
blankcorrect_table['CLIENT_ID'] = blankcorrect_table['CLIENT_ID'].cat.set_categories(finclip['CLIENT_ID'])
blankcorrect_table.sort_values(['CLIENT_ID'], inplace = True)
blankcorrect_table.reset_index(drop=True, inplace=True)
blankcorrect_table

### lipid nomalized

In [ ]:
lipid_table = pd.concat([finclip_l, muscle_l], axis = 0)
lipid_table.sort_values(['CLIENT_ID'], inplace = True)
lipid_table.reset_index(drop=True, inplace=True)
lipid_table

In [ ]:
t= lipid_table
width = 0.2
for r in t["CLIENT_ID"].unique():
    plt.figure()
    temp = t.loc[(t.CLIENT_ID == r) & (t.tag == "muscle"), t.columns[2:]].dropna(axis=1)
    x = np.arange(len(temp.columns)) - (width/2)
    plt.xticks(np.arange(len(temp.columns)), temp.columns)
    y = temp.to_numpy("float")[0]

    plt.bar(x,y, width=width, label = "muscle")


    temp = t.loc[(t.CLIENT_ID == r) & (t.tag == "finclip"), temp.columns]
    
    y = temp.to_numpy("float")[0]
    
    plt.bar(x + width,y, width=width, label = "finclip")
    plt.title(f'{r}')
    plt.legend()
    


In [ ]:
t= lipid_table

for r in t["CLIENT_ID"].unique():
    plt.figure()
    temp = t.loc[(t.CLIENT_ID == r) & (t.tag == "muscle"), t.columns[2:]].dropna(axis=1)
    x = np.arange(len(temp.columns)) 
    plt.xticks(np.arange(len(temp.columns)), temp.columns)
    y = temp.to_numpy("float")[0]
    plt.xlabel("fish ID")
    plt.ylabel("pg/g lw")
    plt.scatter(x,y, label = "muscle")
    

    temp = t.loc[(t.CLIENT_ID == r) & (t.tag == "finclip"), temp.columns]
    
    y = temp.to_numpy("float")[0]
    
    plt.scatter(x,y, label = "finclip")
    plt.title(f'{r}')
    plt.legend()

In [ ]:
y